In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [18]:
PROJECT_ID = "qwiklabs-gcp-03-61b7f0159050"  # @param {type:"string"}
LOCATION = "us-west1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [19]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [20]:
model = GenerativeModel("gemini-1.5-flash")

In [4]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [29]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some flower shop names that evoke a sense of permanence, beauty, and nostalgia, perfect for a dried flower specialty shop:

**Evoking Timelessness:**

* **The Everbloom**
* **Dried & Divine**
* **Forever Blooms**
* **The Timeless Bouquet**
* **Wildflower Preserve**
* **Bloom & Wither**

**Focusing on Aesthetics:**

* **The Dusty Rose**
* **Sun-Kissed Florals**
* **The Dried Petal Shop**
* **Paper Flower Studio**
* **Botanicals & Bloom**
* **The Pressed Garden**

**Evoking Nostalgia & Sentiment:**

* **Grandmother's Garden**
* **Whispers of Spring**
* **The Memory Bloom**
* **Timeless Blooms**
* **The Dried Garden**
* **Forever in Bloom**

**Unique and Playful:**

* **The Petal Pantry**
* **Bloom & Co.**
* **The Flower Muse**
* **Dried & Delightful**
* **The Bloom Apothecary**

**When choosing a name, consider:**

* **Your target audience:** Who are you trying to reach?
* **Your brand personality:** Do you want your shop to be whimsical, elegant, or rustic?
* **The availability

✅ Recommended. The prompt below is to the point and concise.

In [30]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Names for a Dried Flower Shop:

**Evoking Timelessness & Nostalgia:**

* **The Dried Bloom**
* **Forever Blooms**
* **Whispers of Time**
* **The Timeless Bouquet**
* **Past & Present Blooms**
* **The Herbarium**
* **The Dried Flower Apothecary**

**Highlighting Beauty & Uniqueness:**

* **Dusty Rose**
* **Sun-Kissed Blooms**
* **Ethereal Flowers**
* **The Dried Muse**
* **Wild & Dried**
* **Botanical Art**
* **The Dried Garden**

**Focusing on Sustainability & Eco-Friendly:**

* **The Sustainable Bloom**
* **The Earthly Bouquet**
* **Eco-Dried Flowers**
* **Nature's Legacy**
* **The Green Petal**
* **Forever Flowers**
* **Rebloom**

**Playful & Whimsical:**

* **The Poppy Patch**
* **The Dried & Delicate**
* **Bloom & Co.**
* **Faded Glory**
* **The Flower Press**
* **The Botanical Bazaar**
* **Bloom & Bloom Again**

**Unique & Modern:**

* **Bloom & Dust**
* **The Dried Bouquet Co.**
* **The Bloom Lab**
* **The Paper Petal**
* **The Dried Bloom Studio**
* **Botanical Alchemy**
* **

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [23]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known place in the universe where life exists. It's a dynamic and fascinating world with a rich history and diverse ecosystems. Here's a glimpse:

**Physical Characteristics:**

* **Size:** Earth is a terrestrial planet with a diameter of 12,742 km (7,926 miles).
* **Composition:** Mostly iron and nickel in the core, with a mantle of silicate rocks and a thin outer crust.
* **Atmosphere:** Composed mainly of nitrogen (78%) and oxygen (21%).
* **Water:** 71% of the Earth's surface is covered in water, forming oceans, lakes, and rivers.
* **Gravity:** Earth's gravitational pull is what keeps us grounded and keeps the atmosphere intact.
* **Magnetic Field:** Earth has a magnetic field that protects us from harmful solar radiation.

**Life and Environment:**

* **Biosphere:** Earth is home to an incredible diversity of life, from single-celled organisms to complex mammals.
* **Ecosystems:** Earth's diverse lands

✅ Recommended. The prompt below is specific and well-defined.

In [24]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## Earth: A Unique Planet in the Solar System

Here's a list highlighting Earth's uniqueness compared to other planets in our solar system:

**1. Life:** Earth is the only known planet to harbor life, with a diverse ecosystem ranging from microscopic bacteria to complex animals.  This is due to a unique combination of factors:
    * **Liquid Water:**  Earth has a significant amount of liquid water on its surface, essential for life as we know it.
    * **Atmosphere:** Earth's atmosphere provides protection from harmful radiation and regulates temperature.
    * **Plate Tectonics:** The Earth's dynamic tectonic plates contribute to a stable climate and create diverse geological features.

**2. Climate and Temperature:**  Earth's temperature range allows for liquid water to exist, crucial for life.  It also experiences a diverse range of climates due to its tilted axis and atmospheric circulation, leading to different seasons and weather patterns.

**3. Magnetic Field:**  Earth's magneti

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [25]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Here's the breakdown:

**Boiling Water:**

* **Best Method:** The best method for boiling water depends on your resources and desired speed.  Here's a breakdown:

    * **Stovetop (Pot/Kettle):** This is the most common and usually fastest way for home use. Use a pot or kettle, fill with water, and heat on a stovetop burner.  
    * **Microwave:** Quick and convenient for small amounts of water. However, it might not get as hot as stovetop boiling. 
    * **Electric Kettle:** Very efficient and fast, especially for larger volumes. 
    * **Camping (Campfire/Stove):**  Use a pot or kettle over a fire or a camping stove. This can take longer but is useful for outdoor situations. 

**Why is the Sky Blue?**

* **Rayleigh Scattering:** The color of the sky is due to a phenomenon called Rayleigh scattering.  Sunlight is a mixture of all colors of the rainbow. When sunlight enters the Earth's atmosphere, it collides with tiny particles like nitrogen and oxygen molecules. 

* **Blue Light Scat

✅ Recommended. The prompts below asks one task a time.

In [26]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There's no single "best" method for boiling water, as the ideal approach depends on your specific needs and resources. Here's a breakdown of common methods and their pros and cons:

**1. Stovetop:**

* **Pros:** Simple, accessible, versatile (works with various pot sizes)
* **Cons:** Can be slow, requires constant monitoring, potential for spills or burns

**2. Electric Kettle:**

* **Pros:** Fast, energy-efficient, convenient, automatic shut-off for safety
* **Cons:** Limited capacity, can be expensive

**3. Microwave:**

* **Pros:** Fast, convenient for small quantities, no heating element required
* **Cons:** Uneven heating, not suitable for large volumes, potential for overheating

**4. Camping Stove (propane or butane):**

* **Pros:** Portable, ideal for outdoor use, convenient for camping or emergencies
* **Cons:** Requires fuel, may be less efficient than other methods

**Factors to consider when choosing the best method:**

* **Volume of water:** For small quantities, a microwa

In [27]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky is blue due to a phenomenon called **Rayleigh scattering**. 

Here's how it works:

1. **Sunlight is made of all colors of the rainbow.**  These colors are different wavelengths of light.
2. **When sunlight enters the Earth's atmosphere, it encounters tiny particles, like nitrogen and oxygen molecules.** These particles are much smaller than the wavelengths of visible light.
3. **These tiny particles scatter the shorter wavelengths of light (blue and violet) more effectively than the longer wavelengths (red and orange).** This scattering is called Rayleigh scattering.
4. **The scattered blue light is then spread across the sky,** making it appear blue to our eyes. 

**Here's a simple analogy:** Imagine shining a flashlight through a dusty room. The dust particles scatter the light, making the room appear brighter and more diffuse. In the same way, the tiny particles in the atmosphere scatter the sunlight, making the sky appear blue.

**Why is the sky not violet?** While violet 

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [28]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To find out what day it is today, please check a calendar or ask a voice assistant. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [31]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan offers a wealth of amazing sights! To best recommend a spot, I need a little more information about what you're interested in seeing. 

For example, are you more drawn to:

* **Art and history:** The Duomo, Sforza Castle, Leonardo da Vinci's Last Supper, Pinacoteca di Brera?
* **Fashion and design:** The Galleria Vittorio Emanuele II, the fashion district, the Museo del Novecento?
* **Food and culture:** The Navigli canals, the vibrant street food scene, the opera at Teatro alla Scala? 

Tell me what interests you most, and I can help you find the perfect sightseeing spot in Milan! 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [32]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers with information about destinations, not personal meal planning. 

Perhaps you could tell me about your trip to Milan, and I can suggest some local restaurants based on your interests!  



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [33]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

It's great you're looking to improve your programming skills! To give you the best recommendation, I need a little more information. Tell me:

**1. What programming language(s) are you familiar with?** (e.g., Python, Java, JavaScript, C++, etc.)
**2. What are your interests?** (e.g., game development, web development, data science, mobile app development, etc.)
**3. What is your current skill level?** (e.g., beginner, intermediate, advanced)
**4. Do you prefer working solo or in a team?**

Once you tell me this, I can provide a tailored recommendation that will be fun and challenging for you.

For example, if you're a beginner in Python and interested in game development, I might suggest making a simple text-based adventure game. Or, if you're more experienced in Java and want to work on a team project, I might recommend participating in a hackathon focused on web development. 

I'm excited to help you on your programming journey! 😊 



#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [35]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

As a high school student, it's great you're interested in learning programming! Here's a breakdown of the options and why you might choose one over the other:

**a) Learn Python:**

* **Pros:**
    * **Beginner-friendly:** Python has a clean, easy-to-read syntax, making it ideal for beginners. 
    * **Versatile:**  Used in web development, data science, machine learning, automation, and more. 
    * **Huge community:** Extensive resources, libraries, and support online.
    * **In-demand:**  Python is a highly sought-after skill in the tech industry.

* **Cons:**
    * **Not as performant as some other languages:**  While Python is generally fast enough for many tasks, it's not the most efficient language for high-performance computing.

**b) Learn JavaScript:**

* **Pros:**
    * **Web development powerhouse:**  JavaScript is the language of the web, essential for interactive websites and web applications. 
    * **Cross-platform:** Can be used to develop applications for both web br

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [36]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [37]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [38]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **positive** 

Here's why:

* **"Something surprised me about this video - it was actually original"**: This phrase expresses a positive feeling. The user was pleasantly surprised, suggesting the video exceeded their expectations. 
* **"It was not the same old recycled stuff that I always see"**: This further emphasizes the positive aspect - the video is unique and refreshing.
* **"Watch it - you will not regret it"**: This is a strong recommendation, indicating the user believes others will enjoy the video as much as they did. 

Even though the tweet doesn't contain explicitly positive words like "amazing" or "fantastic," the overall message is positive due to the surprise, appreciation for originality, and strong recommendation. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.